# Prentice Cox PH model with PCA
Fitting Cox PH model with Prentice weights.

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

import pickle

from weighted_concordance import * 

In [2]:
# pd.set_option('display.max_columns', None,'display.max_rows',20)

In [3]:
from lifelines import CoxPHFitter
from lifelines.utils import datetimes_to_durations

In [4]:
# importing data
X_tr = pd.read_csv("D:/compiled_data/X_tr.csv").set_index("csid")

y = pd.read_csv("D:/compiled_data/y.csv").set_index("csid")

Converting date-times from strings to datetime format.

In [5]:
def date_time_conversion(date):
    return(datetime.strptime(date,"%Y-%m-%dT%H:%M:%SZ"))

In [6]:
# Coverting the times into datetime format
y_tr = y.assign(
    event = y.ep_CKB0020_combined_ep,
    start_time = y.study_date.map(date_time_conversion),
    event_time = y.ep_CKB0020_combined_datedeveloped.map(date_time_conversion)
).drop(columns = y.columns)
y_tr["duration"] = datetimes_to_durations(y_tr["start_time"],y_tr["event_time"])[0]
y_tr["subcohort"] = y["subcohort"]
y_tr = y_tr.drop(columns = ["start_time","event_time"])

## Fitting Model

Convergence not reached if naively fit model to all data:

In [7]:
# creating a case dataframe
cases_prentice_df = X_tr.join(y_tr).query("subcohort == False")

cases_prentice_df = cases_prentice_df.assign(
    # setting events outside subcohort to start just before they occur
    start_time = lambda df: df["duration"] - 0.001,
    # adding appropriate weight
    weight = 1,
    subcohort = False
)

# creating a subcohort dataframe
subcohort_prentice_df = X_tr.join(y_tr).query("subcohort == True")


subcohort_prentice_df = subcohort_prentice_df.assign(
        # if it is a case, the weight should be the same as the subcohort until close to the time of the event. 
        duration = lambda df: np.where(df["event"], df["duration"] - 0.001, df["duration"]), 
        # the events start from the origin
        start_time = 0, 
        event = False,
        weight = 1,
        subcohort = True
    )

case_subcohort_prentice_df = pd.concat([cases_prentice_df,subcohort_prentice_df]).drop(columns = "subcohort")

cph2 = CoxPHFitter()
cph2.fit(case_subcohort_prentice_df, entry_col = "start_time", duration_col = "duration",event_col = "event",weights_col = "weight",robust = True)

C:\Users\thali\anaconda3\lib\site-packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low variance means that the column cancer_site_1.0 completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), ConvergenceWarning)


ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

We use PCA to reduce dimension to remedy convergence issues. Number of components will be selected by cross-validation:

In [10]:
from sklearn.decomposition import PCA
from lifelines.utils import k_fold_cross_validation, concordance_index
from sklearn.model_selection import KFold

More detailed annotation for cross validation code can be found in "Unweighted Cox PH with PCA" file

In [11]:
cox_pca_scores = []
nfolds = 3

for i in range(1,20):
    print(i)
    
    kf = KFold(n_splits=nfolds)
    kf.get_n_splits(X_tr)

    scores = []
    
    for train_index, test_index in kf.split(X_tr):
        X_tr_train, X_tr_test = X_tr.iloc[train_index], X_tr.iloc[test_index]
        y_tr_train, y_tr_test = y_tr.iloc[train_index], y_tr.iloc[test_index]
            
        pca = PCA(n_components = i/20)
        X_tr_train_pca = pd.DataFrame(pca.fit_transform(X_tr_train))
        X_tr_train_pca.index = X_tr_train.index

        # creating a case dataframe
        cases_prentice_df = X_tr_train_pca.join(y_tr_train).query("subcohort == False")

        cases_prentice_df = cases_prentice_df.assign(
            # setting events outside subcohort to start just before they occur
            start_time = lambda df: df["duration"] - 0.001,
            # adding appropriate weight
            weight = 1,
            subcohort = False
        )
    
        # creating a subcohort dataframe
        subcohort_prentice_df = X_tr_train_pca.join(y_tr_train).query("subcohort == True")

        subcohort_prentice_df = subcohort_prentice_df.assign(
                # if it is a case, the weight should be the same as the subcohort until close to the time of the event. 
                duration = lambda df: np.where(df["event"], df["duration"] - 0.001, df["duration"]), 
                # the events start from the origin
                start_time = 0, 
                event = False,
                weight = 1,
                subcohort = True
            )

        case_subcohort_prentice_df = pd.concat([cases_prentice_df,subcohort_prentice_df]).drop(columns = "subcohort")

        cph2 = CoxPHFitter()
        cph2.fit(case_subcohort_prentice_df, entry_col = "start_time", duration_col = "duration",event_col = "event",weights_col = "weight",robust = True)
        
        event_times = y_tr_test["duration"]
        event_observed = y_tr_test["event"]
        
        X_tr_test_pca = pca.transform(X_tr_test)
        
        test_preds = cph2.predict_partial_hazard(X_tr_test_pca)
        scores.append(weighted_concordance(event_times, test_preds, event_observed, 0.0011604684001529089/nfolds))
        
    cox_pca_scores.append(np.mean(scores))
 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [12]:
print(cox_pca_scores)
print("Best variation percentage:",np.where(cox_pca_scores == np.max(cox_pca_scores))[0]/20)

[0.5742394904049702, 0.5742394904049702, 0.5767327777083842, 0.6855417215160036, 0.6932580849543996, 0.6666264318150076, 0.6877038396705873, 0.7182255054707642, 0.6827780584723078, 0.7271178499096678, 0.706688040236104, 0.7273069039882879, 0.7122964381352087, 0.6890665934578935, 0.6774814586369974, 0.545404317118681, 0.6546348118006492, 0.6413076704313297, 0.726558762931978]
Best variation percentage: [0.55]


Fitting final model:

In [25]:
pca = PCA(n_components = 0.55)
X_tr_pca = pd.DataFrame(pca.fit_transform(X_tr))
X_tr_pca.index = X_tr.index

In [26]:
pickle.dump(pca, open('prentice_cox_pca_wc.pkl', 'wb'))

In [27]:
# creating a case dataframe
cases_prentice_df = X_tr_pca.join(y_tr).query("subcohort == False")

cases_prentice_df = cases_prentice_df.assign(
    # setting events outside subcohort to start just before they occur
    start_time = lambda df: df["duration"] - 0.001,
    # adding appropriate weight
    weight = 1,
    subcohort = False
)

# creating a subcohort dataframe
subcohort_prentice_df = X_tr_pca.join(y_tr).query("subcohort == True")


subcohort_prentice_df = subcohort_prentice_df.assign(
        # if it is a case, the weight should be the same as the subcohort until close to the time of the event. 
        duration = lambda df: np.where(df["event"], df["duration"] - 0.001, df["duration"]), 
        # the events start from the origin
        start_time = 0, 
        event = False,
        weight = 1,
        subcohort = True
    )

case_subcohort_prentice_df = pd.concat([cases_prentice_df,subcohort_prentice_df]).drop(columns = "subcohort")

cph2 = CoxPHFitter()
cph2.fit(case_subcohort_prentice_df, entry_col = "start_time", duration_col = "duration",event_col = "event",weights_col = "weight",robust = True)

<lifelines.CoxPHFitter: fitted with 1190 total observations, 595 right-censored observations>

In [28]:
pickle.dump(cph2, open('prentice_cox_wc.pkl', 'wb'))